This is the final project with all of the experiments stripped out.

### load modules

In [67]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline

### import data into Pandas DataFrame

In [68]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()

### Task 1: Select Features

In [69]:
df = df.drop('email_address', axis=1)
df = df.astype(float)

### Task 2: Eliminate Outliers/Missing Data

In [70]:
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df.fillna(0, inplace=True)

### Task 3: Create New Features

In [71]:
df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)

#### sklearn prep: features_list
Put dependent variable at the top

In [72]:
features_list = list(df.columns)
features_list.remove('poi')
features_list.insert(0, 'poi')

#### Udacity prep: change to dictionary

In [73]:
df = df.transpose()
data_dict = df.to_dict()
### Store to my_dataset for easy export below.
my_dataset = data_dict
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
### Run through Udacity's grading script
from tester import test_classifier
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 4: Try a variety of classifiers
Done in other notebook
### Task 5: Tune precision and recall > 0.3

#### Standardize by subtracting mean and dividing by std_dev

In [74]:
from sklearn import preprocessing
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.StandardScaler()

# Create an object to transform the data to fit minmax processor
features_train = min_max_scaler.fit_transform(features_train)
features_test = min_max_scaler.fit_transform(features_test)

#### Try adaboost again

In [76]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, algorithm='SAMME')
test_classifier(clf, my_dataset, features_list, folds = 1000)

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1.0,
          n_estimators=100, random_state=None)
	Accuracy: 0.83873	Precision: 0.35046	Recall: 0.24550	F1: 0.28874	F2: 0.26114
	Total predictions: 15000	True positives:  491	False positives:  910	False negatives: 1509	True negatives: 12090



Almost no difference. I am wondering if my tansformation has done anything at all? 

In [64]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
test_classifier(clf, my_dataset, features_list, folds = 1000)

GaussianNB()
	Accuracy: 0.61633	Precision: 0.21514	Recall: 0.70900	F1: 0.33011	F2: 0.48592
	Total predictions: 15000	True positives: 1418	False positives: 5173	False negatives:  582	True negatives: 7827



In [65]:
from sklearn import svm
clf = svm.LinearSVC()
test_classifier(clf, my_dataset, features_list, folds=1000)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
	Accuracy: 0.73227	Precision: 0.20561	Recall: 0.35200	F1: 0.25959	F2: 0.30812
	Total predictions: 15000	True positives:  704	False positives: 2720	False negatives: 1296	True negatives: 10280



In [66]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',C=100) 
# tried gamma=0.001, C=100.
# kernel = 'linear', kernel='rbf'
test_classifier(clf, my_dataset, features_list, folds=1000)

Got a divide by zero when trying out: SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision or recall may be undefined due to a lack of true positive predicitons.


Had great discussion with Carl. He suggested specifying a number of dimensions for the PCA. Otherwise the algorithm just rotates the data set without reducing the number of dimensions. 

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=2)), ('svm', LinearSVC())]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=2, whiten=False)), ('svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])
	Accuracy: 0.66133	Precision: 0.17387	Recall: 0.41050	F1: 0.24427	F2: 0.32267
	Total predictions: 15000	True positives:  821	False positives: 3901	False negatives: 1179	True negatives: 9099



Great Recall, lousey precision. Try more dimensions, going up to 3. 

In [78]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=3)), ('svm', LinearSVC())]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=3, whiten=False)), ('svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])
	Accuracy: 0.71047	Precision: 0.18818	Recall: 0.35350	F1: 0.24561	F2: 0.30067
	Total predictions: 15000	True positives:  707	False positives: 3050	False negatives: 1293	True negatives: 9950



Getting worse with three dimensions. Both recall and precision are down. I am going to try a different model on the backend. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=2)), ('svm', svm.SVC(kernel='rbf'))]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)